In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

# Extract information from MIT-BIH raw Files

## Get Record Names from the RECORDS File

In [2]:
rlist = []
records = os.path.normpath('mit-bih-raw/RECORDS')
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it

## Extract all info using the WaveForm DataBase (WFDB)

In [3]:

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
            # Then replace _____ with the correct function call
        samp = wfdb.rdsamp(os.path.normpath('mit-bih-raw/'+x)) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
        ######################################################
        samples.append(samp) #add it to our array for all subject
        
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann(os.path.normpath('mit-bih-raw/'+x), extension="qrs") #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann(os.path.normpath('mit-bih-raw/'+x),extension="atr") #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

  0%|                                                    | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|███████████████████████████████████████████| 25/25 [00:30<00:00,  1.23s/it]


In [4]:
for sample in qrs_locs:
    print(sample)
# Now, in this code block use wfdb to extract the sample info, QRS info, and atr info
# Print some stuff out and see if you can figure out how to manipulate it

193
390
586
782
905
1179
1377
1578
1778
1977
2175
2374
2575
2773
2971
3172
3374
3576
3783
3991
4197
4408
4619
4828
5038
5251
5463
5674
5883
6085
6285
6486
6686
6882
7071
7264
7458
7652
7845
8041
8238
8435
8631
8829
9028
9225
9423
9622
9822
10020
10216
10417
10617
10816
11015
11219
11424
11627
11835
12043
12250
12455
12662
12866
13071
13277
13483
13686
13890
14095
14297
14501
14707
14915
15122
15328
15533
15738
15937
16137
16340
16543
16745
16950
17154
17357
17562
17768
17971
18172
18373
18576
18775
18978
19186
19393
19599
19805
20012
20214
20418
20622
20821
21024
21227
21423
21622
21821
22018
22215
22416
22618
22819
23020
23223
23425
23626
23830
24033
24233
24433
24633
24835
25035
25234
25435
25633
25834
26035
26237
26439
26641
26846
27045
27244
27445
27640
27837
28036
28232
28435
28639
28841
29042
29244
29442
29637
29837
30038
30234
30433
30633
30828
31025
31221
31415
31610
31808
32008
32209
32409
32613
32817
33019
33221
33426
33631
33835
34040
34245
34451
34657
34863
35068
35269
3547

279776
279978
280177
280375
280569
280760
280953
281147
281343
281534
281730
281926
282119
282314
282511
282702
282897
283092
283288
283481
283673
283867
284064
284260
284457
284658
284855
285053
285252
285445
285642
285837
286032
286226
286420
286614
286806
287000
287197
287391
287585
287781
287980
288180
288377
288573
288771
288965
289161
289308
289503
289698
289896
290091
290285
290479
290674
290868
291074
291273
291471
291666
291864
292060
292254
292449
292645
292840
293035
293234
293429
293621
293817
294011
294203
294396
294592
294790
294900
295182
295378
295571
295762
295953
296147
296342
296534
296731
296931
297127
297327
297529
297732
297934
298137
298341
298546
298750
298951
299154
299355
299552
299748
299941
300137
300333
300530
300724
300919
301118
301314
301509
301707
301906
302103
302303
302505
302710
302912
303114
303317
303521
303718
303914
304112
304312
304510
304708
304907
305105
305300
305493
305687
305880
306073
306270
306467
306661
306857
307053
307246
307440
307635

670312
670503
670696
670887
671082
671277
671468
671661
671856
672047
672237
672430
672625
672820
673015
673212
673411
673606
673801
673997
674194
674387
674579
674773
674886
675162
675356
675547
675740
675931
676124
676319
676512
676705
676903
677101
677298
677497
677696
677895
678092
678290
678485
678679
678872
679066
679259
679445
679634
679824
680015
680203
680394
680585
680773
680965
681159
681349
681542
681737
681929
682121
682316
682508
682699
682894
683089
683283
683480
683676
683870
684065
684261
684455
684648
684843
685039
685234
685427
685619
685811
686005
686195
686384
686578
686773
686968
687162
687358
687555
687749
687946
688143
688338
688536
688736
688934
689128
689323
689519
689710
689901
690096
690289
690480
690672
690862
691052
691244
691437
691628
691821
692018
692216
692410
692604
692799
692991
693183
693376
693567
693754
693943
694135
694326
694516
694710
694904
695099
695295
695489
695682
695872
696064
696258
696450
696644
696839
697032
697226
697421
697614
697805

1003775
1003968
1004163
1004353
1004542
1004732
1004919
1005107
1005296
1005486
1005673
1005862
1006051
1006243
1006433
1006622
1006813
1007005
1007195
1007388
1007582
1007773
1007964
1008157
1008347
1008536
1008727
1008916
1009022
1009296
1009489
1009677
1009866
1010058
1010250
1010441
1010632
1010821
1010929
1011202
1011394
1011582
1011773
1011964
1012155
1012346
1012539
1012730
1012922
1013117
1013313
1013508
1013702
1013899
1014095
1014288
1014482
1014674
1014863
1015055
1015247
1015439
1015631
1015825
1016017
1016205
1016398
1016590
1016778
1016968
1017160
1017353
1017546
1017736
1017928
1018038
1018312
1018504
1018694
1018880
1019068
1019260
1019451
1019638
1019827
1020019
1020211
1020402
1020593
1020784
1020973
1021162
1021351
1021543
1021732
1021920
1022110
1022301
1022490
1022681
1022871
1023061
1023252
1023445
1023637
1023822
1024008
1024192
1024379
1024568
1024753
1024941
1025132
1025323
1025513
1025704
1025897
1026087
1026277
1026469
1026661
1026853
1027048
1027242
1027438


1323337
1323527
1323716
1323910
1324105
1324298
1324492
1324688
1324882
1325074
1325170
1325266
1325455
1325642
1325832
1326020
1326206
1326393
1326581
1326765
1326953
1327144
1327334
1327523
1327712
1327901
1328089
1328278
1328468
1328660
1328849
1329038
1329229
1329419
1329549
1329807
1329998
1330184
1330373
1330564
1330755
1330945
1331137
1331329
1331521
1331711
1331901
1332092
1332283
1332473
1332667
1332861
1333051
1333243
1333439
1333633
1333824
1334017
1334211
1334402
1334592
1334782
1334972
1335158
1335346
1335534
1335723
1335909
1336094
1336281
1336466
1336653
1336842
1337032
1337220
1337411
1337606
1337796
1337983
1338170
1338357
1338545
1338726
1338913
1339099
1339285
1339469
1339656
1339844
1340030
1340217
1340408
1340599
1340789
1340982
1341175
1341364
1341554
1341746
1341934
1342122
1342312
1342505
1342693
1342881
1343068
1343255
1343442
1343627
1343815
1344002
1344184
1344369
1344557
1344745
1344930
1345117
1345229
1345494
1345684
1345870
1346051
1346234
1346420
1346606


...

7771741
7771861
7772054
7772171
7772268
7772385
7772489
7772622
7772884
7773090
7773206
7773361
7773480
7773623
7773866
7774082
7774220
7774343
7774501
7774661
7774879
7775000
7775169
7775360
7775635
7775868
7775988
7776155
7776307
7776474
7776648
7776820
7776936
7777057
7777158
7777259
7777434
7777556
7777669
7777834
7777977
7778169
7778352
7778540
7778770
7779178
7779398
7779579
7779740
7779910
7780158
7780308
7780440
7780626
7780866
7781051
7781240
7781435
7781579
7781710
7781875
7782069
7782203
7782365
7782506
7782630
7782792
7783021
7783135
7783188
7783488
7783643
7783900
7784053
7784279
7784398
7784515
7784674
7784806
7784980
7785089
7785188
7785305
7785469
7785665
7785800
7786009
7786228
7786396
7786571
7786746
7786882
7787102
7787261
7787390
7787506
7787704
7787824
7788048
7788236
7788421
7788645
7788828
7789064
7789185
7789325
7789557
7789676
7789772
7789873
7790079
7790194
7790301
7790441
7790658
7790806
7790961
7791312
7791525
7791735
7791949
7792111
7792298
7792433
7792672


8060293
8060514
8060635
8060825
8060987
8061227
8061380
8061599
8061728
8061912
8062125
8062313
8062450
8062661
8062834
8062993
8063104
8063266
8063424
8063585
8063767
8063968
8064104
8064234
8064345
8064466
8064600
8064837
8065022
8065215
8065414
8065556
8065668
8065846
8066073
8066313
8066483
8066677
8066800
8066983
8067272
8067462
8067687
8067936
8068186
8068390
8068535
8068666
8068802
8068948
8069185
8069322
8069492
8069691
8069926
8070129
8070271
8070394
8070693
8070926
8071062
8071191
8071344
8071489
8071658
8071801
8071918
8072053
8072223
8072441
8072736
8072889
8073096
8073226
8073431
8073564
8073712
8073838
8074076
8074210
8074360
8074506
8074620
8074830
8075020
8075246
8075365
8075545
8075691
8075802
8075965
8076082
8076289
8076474
8076603
8076721
8076842
8077020
8077198
8077309
8077542
8077725
8077888
8078157
8078316
8078456
8078574
8078737
8078980
8079131
8079329
8079456
8079569
8079721
8079972
8080124
8080236
8080354
8080539
8080668
8080786
8080934
8081054
8081158
8081282


8374399
8374546
8374770
8374889
8375050
8375293
8375448
8375632
8375805
8375961
8376217
8376377
8376491
8376614
8376864
8377171
8377314
8377462
8377839
8378130
8378257
8378474
8378620
8378779
8379080
8379233
8379446
8379603
8379767
8379948
8380076
8380206
8380327
8380531
8380727
8380930
8381100
8381315
8381434
8381607
8381893
8382035
8382196
8382380
8382575
8382715
8382834
8383071
8383244
8383359
8383550
8383771
8383959
8384117
8384235
8384354
8384520
8384730
8384879
8385012
8385117
8385271
8385419
8385530
8385717
8385840
8386043
8386249
8386474
8386621
8386742
8386947
8387138
8387381
8387521
8387671
8387851
8388013
8388119
8388301
8388520
8388677
8388864
8389035
8389245
8389366
8389475
8389625
8389784
8389971
8390091
8390229
8390406
8390598
8390808
8390957
8391216
8391383
8391484
8391706
8391846
8391983
8392124
8392278
8392490
8392630
8392775
8392951
8393060
8393198
8393333
8393495
8393680
8393849
8394025
8394252
8394551
8394747
8394884
8395020
8395238
8395447
8395635
8395753
8395963


8728397
8728543
8728654
8728771
8728915
8729055
8729193
8729306
8729511
8729715
8729833
8730018
8730204
8730413
8730620
8730804
8730926
8731049
8731224
8731372
8731623
8731765
8731943
8732071
8732174
8732324
8732573
8732789
8732909
8733114
8733329
8733490
8733627
8733825
8734036
8734176
8734365
8734646
8734845
8734988
8735141
8735337
8735488
8735607
8735721
8735950
8736147
8736279
8736383
8736595
8736728
8736924
8737073
8737293
8737465
8737670
8737871
8738119
8738347
8738473
8738692
8738978
8739104
8739213
8739507
8739801
8740075
8740214
8740410
8740600
8740721
8740832
8740945
8741086
8741248
8741388
8741505
8741707
8741887
8742057
8742180
8742294
8742549
8742689
8742816
8742935
8743048
8743202
8743396
8743575
8743805
8743979
8744188
8744407
8744532
8744802
8744942
8745063
8745170
8745329
8745442
8745593
8745854
8746070
8746298
8746538
8746675
8746818
8746953
8747190
8747439
8747622
8747826
8748042
8748258
8748394
8748522
8748731
8748877
8749033
8749256
8749377
8749486
8749683
8749819


8977635
8977857
8977994
8978103
8978210
8978332
8978476
8978610
8978833
8979071
8979248
8979384
8979597
8979717
8979825
8979894
8980060
8980230
8980352
8980457
8980583
8980764
8980890
8981013
8981163
8981318
8981548
8981680
8981809
8981939
8982043
8982215
8982356
8982577
8982694
8982802
8982928
8983052
8983161
8983283
8983395
8983548
8983677
8983865
8983981
8984124
8984268
8984452
8984609
8984730
8984859
8984972
8985103
8985250
8985407
8985587
8985695
8985804
8986020
8986143
8986258
8986406
8986526
8986646
8986769
8986879
8986996
8987120
8987418
8987555
8987732
8988166
8988314
8988429
8988547
8988703
8988880
8989042
8989189
8989354
8990235
8990296
8990569
8990700
8990919
8990974
8991052
8991172
8991254
8991466
8991524
8991609
8991661
8991771
8991872
8991970
8992021
8992148
8992247
8992352
8992520
8992586
8992761
8992853
8992993
8993061
8993130
8993285
8993395
8993534
8993636
8993729
8993799
8993962
8994021
8994163
8994258
8994346
8994396
8994468
8994557
8994609
8994711
8994763
8994855


### Extracting Rhythm Data

Next, I am going to reformat the rhythm annotations into a different format that is more understandable.

The current format for an individuals rhythm annotations are as follows:
- `labels = ['(N', '(AFIB', '(N', '(O', ...]`
- `locs   = [  10,    1000, 1234, 1983, ...]`

Where the labels' corresponding locations are where that rhythm begins.

The below code changes it to the following format instead using the python data type Dictionary.

```python
rhythm_annotations = {
    '(N':    [ [10,   999],
               [1234, 1982]
             ], 
    '(AFIB': [ [1000, 1233]
             ],
    '(O':    [ [1983, ...]
             ]    
}
```


This data is now formatted in a 2-Dimensional array in which each pair of values represents a range of values in which a specific rythm is present. 

The data can be accessed like so: 
```python
  rhythm_annotations['(N']         = [ [10,   999],
                                       [1234, 1982]
                                     ]
    
  rhythm_annotations['(N'][0]      = [10,   999]

  rhythm_annotations['(N'][0][0]   = 10

```

In [4]:
atr_dics = [] #Initialize the array that will hold the dictionary for each subject

for idxs,lab in enumerate(atr_label):
    atr_dic = {} #Initialize dictionary for each subject
    for idx,x in enumerate(lab):
        if x not in atr_dic.keys():
            atr_dic[x] = [] #Add dictionary key if does not exist
        atr_dic[x].append([atr_locs[idxs][idx], atr_locs[idxs][idx+1]]) #Insert range for each rhythm
    atr_dics.append(atr_dic) #Add to dictionary array

## Other Ways To Format

Here we are going to format each subjects data into a DataFrame using pandas. Many statistical tools are built to be used with DataFrames. 

Also, it allows for a one-stop shop for our data where we can save all data for each subject in one file instead of having multiple files per subject.

Our data frame is going to be formatted like so:

|     | Signal 1 | Signal 2 | R-Peak | Normal | AFIB  | Other |
|-----|----------|----------|--------|--------|-------|-------|
| ... | ...      | ...      | ...    | ...    | ...   | ...   |
| 234 | 0.123    | -0.312   | True   | True   | False | False |
| ... | ...      | ...      | ...    | ...    | ...   | ...   |


- Column 1: Index
    - the index is the value of each row and represents the sample value
- Column 2: Signal 1
    - a float (or decimal) value which represents the value of the first signal in the reading at the given sample value
- Column 3: Signal 2
    - a float (or decimal) value which represents the value of the second signal in the reading at the given sample value
- Column 4: R-Peak
    - a boolean value (```True``` or ```False```) which represents if there is a R-Peak at the given sample value
- Column 5: Normal
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of Normal beats
- Column 6: AFIB
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of AFIB beats
- Column 7: Other
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of other beats

In [12]:
subject_dataframes = [] # Initialize the subject_dataframes - will hold all of our subject dataframes

for s, _ in enumerate(tqdm(good_list)): # Iterate through all of the subjects that we have complete data of 
    subj = pd.DataFrame( # The below statements initialize our datafram. The first to columns will be our given signals, and the rest we initialize to 0
        data = np.transpose(np.array([ # First we give our data, for pandas they want the data by row instead of by column, so we use transpose to get the proper format
                                               [x[0] for x in samples[s][0]],
                                               [x[1] for x in samples[s][0]],
                                               np.zeros(len(samples[s][0])), # np.zeros makes an array of zeros with the given lenth
                                               np.zeros(len(samples[s][0])), 
                                               np.zeros(len(samples[s][0])), 
                                               np.zeros(len(samples[s][0])), 
                                        ])
                           ),
        columns = ['Signal 1', 'Signal 2', 'R-Peak', 'Normal', 'AFIB', 'Other'] # Here we name our columns to match the dataframe we outlined above
    )
    norm = [] # Initialize the norm array which will list every index the person is in a normal rhythm
    if '(N' in atr_dics[s].keys():
        for x in atr_dics[s]['(N']: # Then we iterate through our ranges we extracted above
            norm = norm + list(range(x[0], x[1])) # And add all values in the range to our norm array
    af = [] # Then we do the same steps above for AFIB rhythms
    if '(AFIB' in atr_dics[s].keys():
        for x in atr_dics[s]['(AFIB']:
            af = af + list(range(x[0], x[1]))
    subj['R-Peak']= subj.index.isin(qrs[s]) # the isin() function of a DataFram index will return true if the index is in that list and false if it is not
                                            # then, we can initialize our dataFrame with correct values based on that
    subj['Normal']= subj.index.isin(norm)
    subj['AFIB'] = subj.index.isin(af)
    subj['Other'] = ~subj.index.isin(np.append(norm, af)) # Because we are classifying AFIB specifically we define other as any rhythm not in the norm or AFIB list
    
    subject_dataframes.append(subj) # Add the dataframe we built to our to array that holds all of our subjects' dataframes

100%|███████████████████████████████████████████| 23/23 [02:06<00:00,  5.49s/it]


In [13]:
for df in subject_dataframes:
    print(df['AFIB'].value_counts())
    df['AFIB'].value_counts()

False    9146451
True       59309
Name: AFIB, dtype: int64
False    7225834
True     1979926
Name: AFIB, dtype: int64
False    9115566
True       90194
Name: AFIB, dtype: int64
False    8861267
True      344493
Name: AFIB, dtype: int64
True     4888252
False    4317508
Name: AFIB, dtype: int64
False    8435147
True      770613
Name: AFIB, dtype: int64
True     6640174
False    2565586
Name: AFIB, dtype: int64
False    9184068
True       21692
Name: AFIB, dtype: int64
True     5802236
False    3403524
Name: AFIB, dtype: int64
False    9085887
True      119873
Name: AFIB, dtype: int64
True     8778384
False     427376
Name: AFIB, dtype: int64
False    8232180
True       92820
Name: AFIB, dtype: int64
False    4866381
True     4339379
Name: AFIB, dtype: int64
True     9205586
False        174
Name: AFIB, dtype: int64
True     9205717
False         43
Name: AFIB, dtype: int64
True     5550672
False    3655088
Name: AFIB, dtype: int64
False    7738455
True     1467305
Name: AFIB, dtype: int

## Saving Extracted Information 

Saving the information that we have used processing power to extract is important because:
1. It makes our data easier to access in the future
    - Easy access in new files
2. It creates static information for us to use and reference
3. By saving in in a CSV we make it more accessible for others to use
    - The data can now be used in an excel sheet and more
    
We will generally always extract to a CSV file unless the data is too complex. If that is the case then we have another option. 

'pickle' is a Python package which will save much more complex Data types for future use. 

For example - if you have want to save a statistical model, pickle will be able to do that more effectively than CSV format.

In [17]:
reload_flag = False
  # Set this flag to true to re-save all of the extracted information even if it has already been saved at these paths

In [18]:
for idx, x in enumerate(tqdm(good_list)): 
    if not os.path.exists('mit-bih-dataframes/'+x+ '.csv') or reload_flag:
        subject_dataframes[idx].to_csv(os.path.normpath('mit-bih-dataframes/'+x+'.csv')) # Pandas DataFrames have a built in to_csv() function which whill save it at the passed path

np.savetxt(os.path.normpath("mit-bih-dataframes/subject_list.csv"), good_list, delimiter=",",  fmt='%s') 
   # We'll load the complete list of subjects as well so that we can easily recreate the file names

100%|███████████████████████████████████████████| 23/23 [08:54<00:00, 23.23s/it]


In [19]:
np.savetxt("mit-bih-extracted/subject_list.csv", good_list, delimiter=",",  fmt='%s') #Save the names in the folder 
for idx, x in enumerate(tqdm(good_list)): # Iterate through our subjects
    if not os.path.exists("mit-bih-extracted/"+x+"_signals.csv") or reload_flag:
        np.savetxt(os.path.normpath("mit-bih-extracted/"+x+"_signals.csv"), np.array(samples[idx][0]), delimiter=",") # numPy has a savetxt() function which by setting the delimiter as ',' we can 
                                                                                            # simulate a to_csv() function 
    if not os.path.exists("mit-bih-extracted/"+x+"_rpeaks.csv") or reload_flag:
            np.savetxt(os.path.normpath("mit-bih-extracted/"+x+"_rpeaks.csv"), np.array(qrs[idx]), delimiter=",")      
    if not os.path.exists("mit-bih-extracted/"+x+"_headers.pkl") or reload_flag:
        with open(os.path.normpath("mit-bih-extracted/"+x+"_headers.pkl"), 'wb') as picklefile: # nomPy has no way to save a dictionary as a CSV so we use the pickle package
                                    # First we open up the file we would like to write to
            pickle.dump(samples[idx][1], picklefile)
    if not os.path.exists("mit-bih-extracted/"+x+"_rhythms.pkl") or reload_flag:
        with open(os.path.normpath("mit-bih-extracted/"+x+"_rhythms.pkl"), 'wb') as picklefile:
            pickle.dump(atr_dics[idx], picklefile)


100%|███████████████████████████████████████████| 23/23 [06:07<00:00, 15.98s/it]
